In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/train.csv')
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Our CRAZY machine learning thing :) ...

In [3]:
from nn_model import *
from preprocessing import *

In [4]:
tX

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [5]:
preprocessing = Preprocessing()

In [6]:
res = preprocessing.fit(tX)

Training started
Epoch #1: 0.22201243592746736
Epoch #2: 0.21857426495223795
Epoch #3: 0.21848868340337324
Epoch #4: 0.21854448717266478
Epoch #5: 0.21854075166433065
Epoch #6: 0.2186110234472979
Epoch #7: 0.21850188210118898
Epoch #8: 0.2185207283078407
Epoch #9: 0.21849563303128408
Epoch #10: 0.2184373481317981
Epoch #11: 0.21858259571554406
Epoch #12: 0.21843611621999176
Epoch #13: 0.21850032087039978
Epoch #14: 0.21847182296253032
Epoch #15: 0.21848372764075547
Epoch #16: 0.21848967361073385
Epoch #17: 0.21847840192768173
Epoch #18: 0.21862791923214686
Epoch #19: 0.21863740794656406
Epoch #20: 0.21853328374183964
Epoch #21: 0.21855715599706857


KeyboardInterrupt: 

In [7]:
training_data = res
training_data

array([[ 0.53470006,  0.26307388,  0.66129922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.9248832 ,  0.67249474,  0.78899839, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.7644879 ,  2.31758719,  1.26104329, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.17205068,  0.48258021,  0.05709405, ...,  1.        ,
         0.        ,  0.        ],
       [-0.44437654, -0.76905141, -0.17366945, ...,  0.        ,
         0.        ,  0.        ],
       [-0.10894809,  0.76029122, -0.10503786, ...,  0.        ,
         0.        ,  0.        ]])

In [8]:
y[y==-1] = 0
y = y.astype(np.int)

In [31]:
model = NN_Model(training_data.shape[1], loss='BinaryCrossEntropy')
model.add_layer(1, activation='sigmoid')
lr = 0.1
lambda_= 0.1
batch_size = 64
epochs = 100
verbose = 1

In [32]:
from nn_model import *

In [33]:
model.train(training_data, y, lr=lr, lambda_=lambda_, batch_size=batch_size, epochs=epochs, verbose=verbose)

Training started
Epoch #1: 45.50919458234908
Epoch #2: 45.131599442167506
Epoch #3: 45.12718639793565
Epoch #4: 45.1104113040697
Epoch #5: 45.12494834038129
Epoch #6: 45.12188030445614
Epoch #7: 45.12258552762705
Epoch #8: 45.13926408297983
Epoch #9: 45.13211102735856
Epoch #10: 45.12682086408023
Epoch #11: 45.116299035699726
Epoch #12: 45.136760341975624
Epoch #13: 45.12381502243245
Epoch #14: 45.13085006268398
Epoch #15: 45.12993648696284
Epoch #16: 45.114277543771514
Epoch #17: 45.12253757634318
Epoch #18: 45.12133769849189
Epoch #19: 45.14580256705363
Epoch #20: 45.126866201689275
Epoch #21: 45.12579602232756
Epoch #22: 45.129539444273
Epoch #23: 45.129556812776926
Epoch #24: 45.129930703639545
Epoch #25: 45.12966842931651
Epoch #26: 45.11928447288094
Epoch #27: 45.128568427978
Epoch #28: 45.13828349482465
Epoch #29: 45.13494982937864
Epoch #30: 45.13157025119625
Epoch #31: 45.13380296735241
Epoch #32: 45.129284037820156
Epoch #33: 45.123788355011115
Epoch #34: 45.13244612611784
Ep

KeyboardInterrupt: 

In [14]:
pred = model.predict(training_data)

In [16]:
res = pred>0.5

In [20]:
np.sum(y) / len(y)

0.342668

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/test.csv')
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)